In [1]:
%load_ext autoreload
%autoreload 2
import time
import multiprocessing
from pathlib import Path
import shutil
import numpy as np
import thor.data as data
import thor.data.dispatch as dispatch
import thor.grid as grid
import thor.option as option
import thor.track as track
import thor.analyze as analyze
import thor.parallel as parallel
import thor.visualize as visualize
import thor.log as log

notebook_name = "gridrad_demo.ipynb"


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



In [2]:
# Parent directory for saving outputs
base_local = Path.home() / "THOR_output"
year=2010
event_directories = data.gridrad.get_event_directories(year, base_local=base_local)
event_directory = event_directories[1]
start, end, event_start = data.gridrad.get_event_times(event_directory)
start = "2010-01-21T18:30:00"
end = "2010-01-21T18:50:00"

period = parallel.get_period(start, end)
intervals = parallel.get_time_intervals(start, end, period=period)

output_parent = base_local / f"runs/dev/gridrad_{event_start.replace('-', '')}"
event_start

'2010-01-21'

In [3]:
if output_parent.exists():
    shutil.rmtree(output_parent)
options_directory = output_parent / "options"

# Create the data_options dictionary
converted_options = {"save": False, "load": False, "parent_converted": None}
gridrad_options = data.gridrad.gridrad_data_options(
    start=start, end=end, converted_options=converted_options, event_start=event_start
)
lon_range = [-102, -89]
lat_range = [27, 39]
era5_pl_options = data.era5.data_options(
    start=start, end=end, longitude_range=lon_range, latitude_range=lat_range
)
args_dict = {"start": start, "end": end, "data_format": "single-levels"}
args_dict.update({"longitude_range": lon_range, "latitude_range": lat_range})
era5_sl_options = data.era5.data_options(**args_dict)

data_options = option.consolidate_options(
    [gridrad_options, era5_pl_options, era5_sl_options]
)
# for testing, ignore the era5 data
data_options = option.consolidate_options([gridrad_options])

dispatch.check_data_options(data_options)
data.option.save_data_options(data_options, options_directory=options_directory)
gridrad_options = data_options["gridrad"]

# Create the grid_options dictionary using the first file in the cpol dataset
grid_options = grid.create_options(
    name="geographic", regrid=False, altitude_spacing=None, geographic_spacing=None
)
grid.check_options(grid_options)
grid.save_grid_options(grid_options, options_directory=options_directory)

# Create the track_options dictionary
track_options = option.default_track_options(dataset="gridrad")
# Modify the default options for gridrad. Because grids so large we now use a distinct
# global flow box for each object.
track_options.levels[1].objects[0].tracking.global_flow_margin = 70
track_options.levels[1].objects[0].tracking.unique_global_flow = False
# If testing, remove the profile and tag attributes
track_options.levels[1].objects[0].attributes["mcs"].pop("profile")
track_options.levels[1].objects[0].attributes["mcs"].pop("tag")
track_options.to_yaml(options_directory / "track.yml")

# Create the display_options dictionary
# visualize_options = {
#     obj: visualize.option.runtime_options(obj, save=True, style="presentation")
#     for obj in ["mcs"]
# }
visualize_options = None

2024-10-28 16:25:13,025 - thor.grid - WARNING - Shape not specified. Will attempt to infer from input.


In [13]:
data_options["gridrad"]["obs_thresh"]

2

In [6]:
Path(f"{output_parent}.tar.gz").unlink(missing_ok=True)

PosixPath('/home/ewan/THOR_output/runs/dev/gridrad_20100121')

In [14]:
%load_ext memory_profiler

In [4]:
times = data.utils.generate_times(data_options["gridrad"])
args = [times, data_options.copy(), grid_options.copy()]
args += [track_options.model_copy(), visualize_options]
track.simultaneous_track(*args, output_directory=output_parent)

2024-10-28 16:25:19,880 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100121.
2024-10-28 16:25:19,881 - thor.track - INFO - Beginning simultaneous tracking.


2024-10-28 16:25:19,975 - thor.track - INFO - Processing 2010-01-21T18:30:00.
2024-10-28 16:25:19,977 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T18:30:00.
2024-10-28 16:25:19,978 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T183000Z.nc
2024-10-28 16:25:35,688 - thor.track - INFO - Processing hierarchy level 0.
2024-10-28 16:25:35,689 - thor.track - INFO - Tracking convective.
2024-10-28 16:26:14,210 - thor.track - INFO - Tracking middle.
2024-10-28 16:26:18,784 - thor.track - INFO - Tracking anvil.
2024-10-28 16:26:21,108 - thor.track - INFO - Processing hierarchy level 1.
2024-10-28 16:26:21,109 - thor.track - INFO - Tracking mcs.
2024-10-28 16:26:31,754 - thor.match.match - INFO - Matching mcs objects.
2024-10-28 16:26:31,802 - thor.match.match - INFO - No previous mask, or no objects in previous mask.
2024-10-28 16:26:32,074 - thor.track - INFO - Processing 2010-01-21T18:40:00.
2024-10-28 16:26:32,075 - thor.data.gridrad 

In [29]:
# num_processes = int(0.75 * os.cpu_count())
# num_processes = os.cpu_count()
num_processes = 2
kwargs = {"initializer": parallel.initialize_process, "processes": num_processes}
with log.logging_listener(), multiprocessing.get_context("spawn").Pool(**kwargs) as pool:
    results = []
    for i, time_interval in enumerate(intervals):
        time.sleep(1)
        args = [i, time_interval, data_options.copy(), grid_options.copy()]
        args += [track_options.model_copy(), visualize_options]
        args += [output_parent, "gridrad"]
        args = tuple(args)
        results.append(pool.apply_async(parallel.track_interval, args))
    pool.close()
    pool.join()
    parallel.check_results(results)


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



2024-10-28 12:13:49,598 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100121/interval_0.
2024-10-28 12:13:49,599 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-28 12:13:49,683 - thor.track - INFO - Processing 2010-01-21T12:00:00.
2024-10-28 12:13:49,686 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T12:00:00.
2024-10-28 12:13:49,686 - thor.data.gridrad - INFO - Converting gridrad data from nexrad_3d_v4_2_20100121T120000Z.nc
2024-10-28 12:13:49,740 - thor.track - INFO - Beginning thor run. Saving output to /home/ewan/THOR_output/runs/dev/gridrad_20100121/interval_1.
2024-10-28 12:13:49,741 - thor.track - INFO - Beginning simultaneous tracking.
2024-10-28 12:13:49,818 - thor.track - INFO - Processing 2010-01-21T13:00:00.
2024-10-28 12:13:49,820 - thor.data.gridrad - INFO - Updating gridrad dataset for 2010-01-21T13:00:00.
2024-10-28 12:13:49,821 - thor.data.gridrad - INFO - Converting gridrad dat

In [30]:
parallel.stitch_run(output_parent, intervals, cleanup=True)

2024-10-28 12:23:50,292 - thor.parallel - INFO - Stitching all attribute, mask and record files.
2024-10-28 12:23:52,299 - thor.parallel - INFO - Stitching record files.
2024-10-28 12:23:52,322 - thor.parallel - INFO - Stitching attribute files.
2024-10-28 12:23:52,760 - thor.parallel - INFO - Stitching mask files.


In [7]:
analysis_options = analyze.mcs.analysis_options()
analyze.mcs.process_velocities(output_parent, profile_dataset=None)
analyze.mcs.quality_control(output_parent, analysis_options)
# analyze.mcs.classify_all(output_parent)

,,convective_contained,anvil_contained,velocity,area,offset,major_axis,axis_ratio,duration
time,universal_id,,,,,,,,
2010-01-21 18:20:00,1,True,True,True,True,True,True,False,False
2010-01-21 18:30:00,1,True,True,True,True,True,True,False,False
2010-01-21 18:40:00,1,True,True,True,True,True,True,True,False


In [8]:
figure_options = visualize.option.horizontal_attribute_options(
    "mcs_velocity_analysis", style="presentation", attributes=["velocity", "offset", "id"]
)
start_time = np.datetime64(start)
end_time = np.datetime64(end)
args = [output_parent, start_time, end_time, figure_options]
args_dict = {"parallel_figure": True, "dt": 7200, "by_date": False, "num_processes": 2}
visualize.attribute.mcs_series(*args, **args_dict)

2024-10-28 15:39:14,737 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-21T18:20:00.000000000.
2024-10-28 15:39:33,555 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 2010-01-21T18:20:00.000000000.



## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119


## You are using the Python ARM Radar Toolkit (Py-ART), an open source
## library for working with weather radar data. Py-ART is partly
## supported by the U.S. Department of Energy as part of the Atmospheric
## Radiation Measurement (ARM) Climate Research Facility, an Office of
## Science user facility.
##
## If you use this software to prepare a publication, please cite:
##
##     JJ Helmus and SM Collis, JORS 2016, doi: 10.5334/jors.119



2024-10-28 15:39:45,579 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-21T18:30:00.000000000.
2024-10-28 15:39:45,905 - thor.visualize.attribute - INFO - Visualizing MCS at time 2010-01-21T18:40:00.000000000.
2024-10-28 15:40:10,433 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 2010-01-21T18:30:00.000000000.
2024-10-28 15:40:10,973 - thor.visualize.attribute - INFO - Saving mcs_velocity_analysis figure for 2010-01-21T18:40:00.000000000.
2024-10-28 15:40:21,021 - thor.visualize.visualize - INFO - Animating mcs_velocity_analysis figures for mcs objects.
2024-10-28 15:40:21,023 - thor.visualize.visualize - INFO - Saving animation to /home/ewan/THOR_output/runs/dev/gridrad_20100121/visualize/mcs_velocity_analysis.gif.
